In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Daily SuperTrend Scheduler (No-CLI version)
- All configuration lives in the CONFIG block below.
- Run this script once per day after market close (IST).
- It persists state to keep tracking positions, pending orders, and ledger.

Usage:
    python daily_supertrend_scheduler_nocli.py
"""
import os, json, math, logging, warnings
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple

import numpy as np
import pandas as pd

try:
    import yfinance as yf
except Exception:
    yf = None

warnings.filterwarnings("ignore", category=FutureWarning)
logging.basicConfig(level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
log = logging.getLogger("st_sched")

# ============================ CONFIG ============================
# Set everything here (no command-line flags)
@dataclass
class Config:
    # Dates
    start_date: str = "2015-01-01"
    end_date: str   = "today"                       # 'today' resolves to Asia/Kolkata today

    # Paths
    universe_path: str = "nifty500.txt"
    cache_dir: str     = "cache"
    out_dir: str       = "daily_outputs"
    state_dir: str     = "state"

    # Best-combo knobs you shared
    min_confirmations: int = 2
    use_macd_confirm: bool = False
    use_ema_confirm: bool = True      # EMA(5/20)
    use_ema921_confirm: bool = False  # EMA(9/21)
    use_adx_confirm: bool = False

    use_rsi_filter: bool = True
    use_bb_reinforce: bool = False
    use_obv_confirm: bool = False

    # Indicators / filters
    rsi_len: int = 14
    rsi_ob_level: float = 70.0
    rsi_require_rising: bool = True
    st_atr_len: int = 10
    st_multiplier: float = 3.0
    macd_fast: int = 12
    macd_slow: int = 26
    macd_signal: int = 9
    macd_cross_lookback: int = 3
    ema_fast: int = 5
    ema_slow: int = 20
    ema_fast_2: int = 9
    ema_slow_2: int = 21
    bb_len: int = 20
    bb_std: float = 2.0
    bb_require_rising: bool = True
    adx_len: int = 14
    adx_min: float = 20.0
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.70

    # Portfolio & execution
    apply_fees: bool = True
    initial_capital: float = 200_000.0
    max_concurrent_positions: int = 4
    deploy_cash_frac: float = 0.25
    top_k_daily: int = 3
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Benchmark preference
    benchmark_try: Tuple[str,...] = ("^CRSLDX","^CNX500","^NIFTY500","^NSEI","^BSESN","^BSE500")

CFG = Config()
# ============================ END CONFIG ============================

TZ = "Asia/Kolkata"
def today_str(): return pd.Timestamp.today(tz=TZ).strftime("%Y-%m-%d")
def resolve_end_date(end: str) -> str: return today_str() if str(end).lower()=="today" else end
def ensure_dirs(*paths): [os.makedirs(p, exist_ok=True) for p in paths]

def load_universe(path: str) -> List[str]:
    if not os.path.exists(path): raise FileNotFoundError(f"universe file not found: {path}")
    out, seen = [], set()
    for ln in open(path, "r", encoding="utf-8"):
        s = ln.strip().upper()
        if not s: continue
        if not s.endswith(".NS"): s += ".NS"
        if s not in seen: out.append(s); seen.add(s)
    return out

def fetch_prices(tickers: List[str], start: str, end: str, cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}
    for t in tickers:
        try:
            df = yf.download(t, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty: continue
            df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"
            data[t] = df
        except Exception as e:
            log.warning("download fail %s: %s", t, e)
    return data

def ema(s, span): return s.ewm(span=span, adjust=False, min_periods=span).mean()
def rsi(s, length=14):
    d=s.diff(); g=(d.where(d>0,0.0)).rolling(length).mean(); l=(-d.where(d<0,0.0)).rolling(length).mean()
    rs=g/l.replace(0.0,np.nan); out=100-(100/(1+rs)); return out.fillna(50.0)
def _tr(h,l,pc): 
    return pd.concat([(h-l).abs(), (h-pc).abs(), (l-pc).abs()], axis=1).max(axis=1)
def atr(h,l,c,length=14): return _tr(h,l,c.shift(1)).rolling(length).mean()
def supertrend(df, atr_len=10, mult=3.0):
    hl2=(df["High"]+df["Low"])/2.0; _a=atr(df["High"],df["Low"],df["Close"],atr_len)
    up=hl2+mult*_a; dn=hl2-mult*_a
    st=pd.Series(index=df.index,dtype=float); dr=pd.Series(index=df.index,dtype=int)
    st.iloc[0]=up.iloc[0]; dr.iloc[0]=1
    for i in range(1,len(df)):
        dr.iloc[i]=1 if df["Close"].iloc[i]>st.iloc[i-1] else (-1 if df["Close"].iloc[i]<st.iloc[i-1] else dr.iloc[i-1])
        st.iloc[i]=min(up.iloc[i],st.iloc[i-1]) if dr.iloc[i]==1 else max(dn.iloc[i],st.iloc[i-1])
        if dr.iloc[i]==1 and df["Close"].iloc[i]<st.iloc[i]: dr.iloc[i]=-1; st.iloc[i]=dn.iloc[i]
        elif dr.iloc[i]==-1 and df["Close"].iloc[i]>st.iloc[i]: dr.iloc[i]=1; st.iloc[i]=up.iloc[i]
    return pd.DataFrame({"st_value":st,"st_dir":dr})
def macd(s,fast=12,slow=26,signal=9):
    ef=s.ewm(span=fast,adjust=False,min_periods=fast).mean(); es=s.ewm(span=slow,adjust=False,min_periods=slow).mean()
    line=ef-es; sig=line.ewm(span=signal,adjust=False,min_periods=signal).mean(); hist=line-sig; return line,sig,hist
def boll(s, length=20, std=2.0):
    mid=s.rolling(length).mean(); dev=s.rolling(length).std(ddof=0); return mid, mid+std*dev, mid-std*dev
def obv(c,v): return (np.sign(c.diff().fillna(0.0))*v).cumsum().fillna(0.0)
def adx_di(h,l,c,length=14):
    ph,pl,pc=h.shift(1),l.shift(1),c.shift(1); up, dn = h-ph, pl-l
    plus_dm=up.where((up>dn)&(up>0),0.0); minus_dm=dn.where((dn>up)&(dn>0),0.0)
    tr=_tr(h,l,pc); alpha=1.0/length
    atr_=tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di=100*(plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean()/atr_)
    minus_di=100*(minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean()/atr_)
    dx=100*(plus_di-minus_di).abs()/(plus_di+minus_di).replace(0,np.nan)
    adx=dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    return adx, plus_di, minus_di

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    d=df.copy()
    st=supertrend(d, cfg.st_atr_len, cfg.st_multiplier); d=pd.concat([d,st],axis=1)
    d["rsi"]=rsi(d["Close"], cfg.rsi_len); d["rsi_prev"]=d["rsi"].shift(1)
    d["ema_f"]=ema(d["Close"], cfg.ema_fast); d["ema_s"]=ema(d["Close"], cfg.ema_slow)
    d["ema_f2"]=ema(d["Close"], cfg.ema_fast_2); d["ema_s2"]=ema(d["Close"], cfg.ema_slow_2)
    d["macd_line"],d["macd_signal"],d["macd_hist"]=macd(d["Close"], cfg.macd_fast, cfg.macd_slow, cfg.macd_signal)
    d["bb_mid"],d["bb_up"],d["bb_dn"]=boll(d["Close"], cfg.bb_len, cfg.bb_std)
    d["obv"]=obv(d["Close"], d["Volume"]); d["obv_prev"]=d["obv"].shift(1)
    d["adx"],d["di_plus"],d["di_minus"]=adx_di(d["High"], d["Low"], d["Close"], cfg.adx_len)
    d["avg_vol_20"]=d["Volume"].rolling(20).mean()
    d["high_52w"]=d["Close"].rolling(cfg.filter_52w_window).max()
    return d.dropna()

def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: str, cache_dir: str):
    for t in benchmarks:
        df_map=fetch_prices([t], start, end, cache_dir); df=df_map.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t); return t, df
    idx=pd.date_range(start=start, end=end, freq="B")
    return "SYNTH", pd.DataFrame({"Close":np.ones(len(idx))}, index=idx)

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str,float]:
    scores={}; b=bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df=data_map.get(t); scores[t]=0.0
        if df is None or df.empty: continue
        if end_dt not in df.index: df=df[df.index<=end_dt]; 
        if df is None or len(df)==0: continue
        r=df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common=pd.concat([r,b],axis=1,keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)): continue
        excess=common["s"]-common["b"]; vol=common["s"].std(ddof=0)
        scores[t]=0.0 if vol<=1e-8 else float((excess.mean()/vol)*math.sqrt(252.0))
    return scores

def markowitz_long_only(mu: np.ndarray, Sigma: np.ndarray) -> np.ndarray:
    n=len(mu); eps=1e-6; Sigma=Sigma+eps*np.eye(n)
    def solve_lambda(lmbd, active=None):
        if active is None:
            A=np.block([[2*lmbd*Sigma, np.ones((n,1))],[np.ones((1,n)), np.zeros((1,1))]]); b=np.concatenate([mu,[1.0]])
            try: sol=np.linalg.solve(A,b); w=sol[:n]
            except np.linalg.LinAlgError: w=np.full(n,1.0/n)
            return w
        idx=np.where(active)[0]
        if len(idx)==0: return np.full(n,1.0/n)
        S=Sigma[np.ix_(idx,idx)]; o=np.ones(len(idx)); m=mu[idx]
        A=np.block([[2*lmbd*S, o[:,None]],[o[None,:], np.zeros((1,1))]]); b=np.concatenate([m,[1.0]])
        try: sol=np.linalg.solve(A,b); wsub=sol[:len(idx)]
        except np.linalg.LinAlgError: wsub=np.full(len(idx),1.0/len(idx))
        w=np.zeros(n); w[idx]=wsub; return w
    best_w=np.full(n,1.0/n); best_sr=-1e9
    for lmbd in np.logspace(-3,3,31):
        active=np.ones(n,dtype=bool); w=None
        for _ in range(n):
            w=solve_lambda(lmbd,active); neg=w<0
            if not neg.any(): break
            worst=np.argmin(w); active[worst]=False
        if w is None: continue
        w=np.clip(w,0,None); 
        if w.sum()<=0: continue
        w=w/w.sum(); mu_p=float(mu@w); vol_p=float(np.sqrt(w@Sigma@w))
        if vol_p<=1e-8: continue
        sr=mu_p/vol_p
        if sr>best_sr: best_sr, best_w = sr, w.copy()
    return best_w

def calc_fees(turnover_buy: float, turnover_sell: float, apply=True) -> float:
    if not apply: return 0.0
    BROKER_PCT=0.001; BROKER_MIN=5.0; BROKER_CAP=20.0
    STT_PCT=0.001; STAMP_BUY_PCT=0.00015; EXCH_PCT=0.0000297; SEBI_PCT=0.000001; IPFT_PCT=0.000001; GST_PCT=0.18
    DP_SELL=20.0 if turnover_sell>=100 else 0.0
    def _br(turn): 
        if turn<=0: return 0.0
        return max(BROKER_MIN, min(turn*BROKER_PCT, BROKER_CAP))
    brb=_br(turnover_buy); brs=_br(turnover_sell)
    stt=STT_PCT*(turnover_buy+turnover_sell); stamp=STAMP_BUY_PCT*turnover_buy
    exch=EXCH_PCT*(turnover_buy+turnover_sell); sebi=SEBI_PCT*(turnover_buy+turnover_sell); ipft=IPFT_PCT*(turnover_buy+turnover_sell)
    gst=GST_PCT*(brb+brs+DP_SELL+exch+sebi+ipft)
    return float((brb+brs)+stt+stamp+exch+sebi+ipft+DP_SELL+gst)

def next_trading_date(current_dt: pd.Timestamp, reference_df: pd.DataFrame) -> Optional[pd.Timestamp]:
    idx=list(reference_df.index)
    for i,d in enumerate(idx):
        if d==current_dt and i+1 < len(idx): return idx[i+1]
        if d>current_dt: return d
    return None

def build_signals_for_date(dt0: pd.Timestamp, data_map: Dict[str,pd.DataFrame], cfg: Config) -> pd.DataFrame:
    out=[]
    for t, df in data_map.items():
        if df is None or df.empty or dt0 not in df.index: continue
        d=compute_indicators(df, cfg); 
        if d.empty or dt0 not in d.index: continue
        row=d.loc[dt0]
        # ST flip up today?
        st_flip_up = (d["st_dir"].shift(1).loc[dt0]==-1) and (row["st_dir"]==1)
        if not st_flip_up: continue
        bits=[]; oks=[]
        if cfg.use_rsi_filter:
            rsi_ok=(row["rsi"]<cfg.rsi_ob_level) and (row["rsi"]>=d["rsi_prev"].loc[dt0] if cfg.rsi_require_rising else True)
            oks.append(rsi_ok); 
            if rsi_ok: bits.append(f"RSI<{cfg.rsi_ob_level:g}"+(" & rising" if cfg.rsi_require_rising else ""))
        if cfg.use_ema_confirm:
            ema_ok=row["ema_f"]>row["ema_s"]; oks.append(ema_ok); 
            if ema_ok: bits.append("EMA5>EMA20")
        if cfg.use_ema921_confirm:
            e921=row["ema_f2"]>row["ema_s2"]; oks.append(e921); 
            if e921: bits.append("EMA9>EMA21")
        if cfg.use_bb_reinforce:
            mid_rise=d["bb_mid"].loc[dt0] >= d["bb_mid"].shift(1).loc[dt0] if cfg.bb_require_rising else True
            bb_ok=(row["Close"]>=row["bb_up"]) and mid_rise; oks.append(bb_ok); 
            if bb_ok: bits.append("Close≥UpperBB & mid rising")
        if cfg.use_obv_confirm:
            obv_ok=row["obv"]>=d["obv_prev"].loc[dt0]; oks.append(obv_ok); 
            if obv_ok: bits.append("OBV rising")
        if cfg.use_adx_confirm:
            adx_ok=(row["adx"]>=cfg.adx_min) and (row["di_plus"]>=row["di_minus"]); oks.append(adx_ok); 
            if adx_ok: bits.append(f"ADX≥{cfg.adx_min:g} & +DI≥−DI")
        confirmed = (sum(1 for v in oks if v) >= int(cfg.min_confirmations)) if (cfg.min_confirmations and cfg.min_confirmations>0) else (all(oks) if oks else True)
        if not confirmed: continue
        # 52w proximity
        hist=df["Close"].loc[:dt0]; window=hist.iloc[-cfg.filter_52w_window:] if len(hist)>=cfg.filter_52w_window else hist
        high_52w=float(window.max())
        if not (high_52w>0 and row["Close"] >= cfg.within_pct_of_52w_high * high_52w): continue
        out.append({"ticker":t,"signal_date":dt0,"close":float(row["Close"]),"reason":"ST flip ↑; "+"; ".join(bits)+f"; min_conf={cfg.min_confirmations}"})
    return pd.DataFrame(out)

def run_once(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir, cfg.state_dir)
    state_path=os.path.join(cfg.state_dir,"portfolio_state.json")
    state={"cash":cfg.initial_capital,"positions":{},"pending_orders":[],"ledger":[]} if not os.path.exists(state_path) else json.load(open(state_path,"r"))
    start=cfg.start_date; end=resolve_end_date(cfg.end_date)
    symbols=load_universe(cfg.universe_path)
    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, start, end, cfg.cache_dir)
    if bench_df is None or bench_df.empty:
        log.error("No benchmark data; abort."); return
    data_map=fetch_prices(symbols, start, end, cfg.cache_dir)
    last_dt=bench_df.index[-1]; prev_dt=bench_df.index[-2] if len(bench_df.index)>=2 else bench_df.index[-1]
    next_dt=next_trading_date(last_dt, bench_df)
    log.info("Calendar: prev=%s last=%s next=%s", str(prev_dt.date()), str(last_dt.date()), str(next_dt.date() if next_dt else None))
    # Execute pending orders for last_dt
    for od in state["pending_orders"]:
        if od.get("exec_date") is None: od["exec_date"] = str(last_dt.date())
    new_pending=[]
    for od in state["pending_orders"]:
        if pd.to_datetime(od["exec_date"]).date() != last_dt.date():
            new_pending.append(od); continue
        t=od["ticker"]; df=data_map.get(t)
        if df is None or df.empty or last_dt not in df.index: new_pending.append(od); continue
        px=float(df.loc[last_dt,"Open"]); side=od["side"]
        if side=="BUY":
            shares=int(od["shares"]); turn=shares*px; fee=calc_fees(turn,0.0,cfg.apply_fees)
            if turn+fee>state["cash"]+1e-6: new_pending.append(od); continue
            state["cash"]-= (turn+fee)
            state["positions"][t]={"entry_date":str(last_dt.date()),"entry_px":px,"shares":shares,"buy_fee":fee,"entry_reason":od.get("entry_reason",""),"entry_signal_reason":od.get("entry_signal_reason","")}
            state["ledger"].append({"date":str(last_dt.date()),"ticker":t,"side":"BUY","price":px,"shares":shares,"turnover":turn,"fees_inr":fee,"reason":od.get("entry_reason",""),"entry_reason":od.get("entry_reason",""),"entry_signal_reason":od.get("entry_signal_reason",""),"exit_reason":"","days_held":0,"pnl_inr":0.0})
            log.info("FILLED BUY  %-10s dt=%s px=%.2f sh=%d fee=%.2f cash→%.2f", t, last_dt.date(), px, shares, fee, state["cash"])
        elif side=="SELL":
            pos=state["positions"].get(t)
            if pos is None: continue
            shares=int(pos["shares"]); turn=shares*px; fee=calc_fees(0.0,turn,cfg.apply_fees)
            pnl=(px-pos["entry_px"])*shares; realized=pnl-fee-pos.get("buy_fee",0.0)
            state["cash"]+=(turn-fee); days=(last_dt.date()-pd.to_datetime(pos["entry_date"]).date()).days
            state["ledger"].append({"date":str(last_dt.date()),"ticker":t,"side":"SELL","price":px,"shares":shares,"turnover":turn,"fees_inr":fee,"reason":od.get("exit_reason",""),"entry_reason":pos.get("entry_reason",""),"entry_signal_reason":pos.get("entry_signal_reason",""),"exit_reason":od.get("exit_reason",""),"days_held":int(days),"pnl_inr":float(realized)})
            del state["positions"][t]
            log.info("FILLED SELL %-10s dt=%s px=%.2f sh=%d pnl=%.2f fee=%.2f cash→%.2f", t, last_dt.date(), px, shares, realized, fee, state["cash"])
    state["pending_orders"]=new_pending
    # Detect exits on last_dt -> plan sell next_dt
    def detect_exits_for_date(dt0):
        planned=[]
        for t,pos in list(state["positions"].items()):
            df=data_map.get(t); 
            if df is None or df.empty or dt0 not in df.index: continue
            d=compute_indicators(df, cfg); 
            if d.empty or dt0 not in d.index: continue
            row=d.loc[dt0]
            st_down = (d["st_dir"].shift(1).loc[dt0]==1) and (row["st_dir"]==-1)
            macd_bear=(row["macd_line"]<row["macd_signal"]) if cfg.use_macd_confirm else False
            ema_bear=(row["ema_f"]<row["ema_s"]) if cfg.use_ema_confirm else False
            ema921_bear=(row["ema_f2"]<row["ema_s2"]) if cfg.use_ema921_confirm else False
            di_bear=(row["di_minus"]>row["di_plus"]) if cfg.use_adx_confirm else False
            rsi_ob=(row["rsi"]>=cfg.rsi_ob_level) if cfg.use_rsi_filter else False
            reason=None
            if st_down: reason="st_bear_flip"
            elif macd_bear: reason="macd_bear"
            elif ema_bear: reason="ema5_20_bear"
            elif ema921_bear: reason="ema9_21_bear"
            elif di_bear: reason="di_bear"
            elif rsi_ob: reason="rsi_overbought"
            if reason and next_dt is not None:
                planned.append({"ticker":t,"side":"SELL","exec_date":str(next_dt.date()),"exit_reason":reason})
        return planned
    sells = detect_exits_for_date(last_dt)
    state["pending_orders"].extend(sells)
    if sells: log.info("Planned SELL next: %s", ", ".join(f"{o['ticker']}({o['exit_reason']})" for o in sells))
    # Build entries on last_dt -> plan buys next_dt
    sig_df = build_signals_for_date(last_dt, data_map, cfg)
    if not sig_df.empty:
        sig_df = sig_df[~sig_df["ticker"].isin(state["positions"].keys())]
    # rank by VOLAᵣ
    if not sig_df.empty:
        volar=compute_volar_scores(last_dt, sig_df["ticker"].tolist(), data_map, bench_df, cfg.volar_lookback)
        sig_df["volar"]=sig_df["ticker"].map(volar); sig_df=sig_df.sort_values("volar", ascending=False).reset_index(drop=True)
    # select
    slots=cfg.max_concurrent_positions - len(state["positions"])
    selected=pd.DataFrame(columns=sig_df.columns)
    if slots>0 and not sig_df.empty:
        selected=sig_df.head(min(cfg.top_k_daily, slots)).copy()
    # weights
    weights=np.array([])
    if not selected.empty:
        names=selected["ticker"].tolist(); rets=[]
        for t in names:
            df=data_map.get(t); ser=df["Close"].loc[:last_dt].pct_change().dropna().iloc[-cfg.volar_lookback:]; rets.append(ser)
        R=pd.concat(rets, axis=1); R.columns=names; R=R.dropna()
        if R.empty or R.shape[0] < max(20, int(0.4*cfg.volar_lookback)) or R.shape[1]==0:
            weights=np.full(len(names), 1.0/len(names))
        else:
            mu=R.mean().values; Sigma=R.cov().values; weights=markowitz_long_only(mu,Sigma)
        weights=weights/weights.sum()
    planned_buys=[]
    if not selected.empty and weights.size>0 and next_dt is not None:
        deploy_cash=max(0.0,float(state["cash"])) * float(cfg.deploy_cash_frac)
        alloc=weights*deploy_cash
        for w_amt, (_, rr) in zip(alloc, selected.iterrows()):
            t=rr["ticker"]; df_t=data_map.get(t)
            last_px=float(df_t.loc[last_dt,"Close"]) if last_dt in df_t.index else float(df_t["Close"].iloc[-1])
            shares=int(math.floor(w_amt/last_px)); 
            if shares<=0: continue
            planned_buys.append({"ticker":t,"side":"BUY","exec_date":str(next_dt.date()),"shares":int(shares),
                                 "entry_reason":"SuperTrend entry","entry_signal_reason":rr["reason"]})
        state["pending_orders"].extend(planned_buys)
    # write outputs
    tag=str(last_dt.date()); ensure_dirs(cfg.out_dir)
    sig_path=os.path.join(cfg.out_dir, f"signals_{tag}.csv"); sel_path=os.path.join(cfg.out_dir, f"selected_{tag}.csv")
    if not sig_df.empty: sig_df.to_csv(sig_path, index=False)
    if not selected.empty:
        sel = selected.copy()
        if weights.size: sel["weight"]=weights; sel["planned_alloc_inr"]=weights*max(0.0,float(state["cash"]))*cfg.deploy_cash_frac
        sel.to_csv(sel_path, index=False)
    pos_path=os.path.join(cfg.out_dir, f"positions_{tag}.csv")
    if state["positions"]:
        pd.DataFrame.from_dict(state["positions"], orient="index").reset_index(names="ticker").to_csv(pos_path, index=False)
    pend_path=os.path.join(cfg.out_dir, f"pending_{tag}.json"); json.dump(state["pending_orders"], open(pend_path,"w"), indent=2)
    ledger_path=os.path.join(cfg.out_dir, "ledger_all.csv")
    if state["ledger"]:
        pd.DataFrame(state["ledger"]).to_csv(ledger_path, index=False)
    json.dump(state, open(state_path,"w"), indent=2)
    log.info("WROTE %s %s %s %s", sig_path if not sig_df.empty else "", sel_path if not selected.empty else "", pos_path if state["positions"] else "", ledger_path if state["ledger"] else "")

def main():
    run_once(CFG)

if __name__=="__main__":
    main()


2025-10-15 22:41:02 | INFO | Using benchmark: ^CRSLDX
2025-10-15 22:43:54 | INFO | Calendar: prev=2025-10-13 last=2025-10-14 next=None
2025-10-15 22:44:37 | INFO | WROTE    
